In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set_style('whitegrid')

%matplotlib inline

In [ ]:
# Parameters and File Paths

# Parameters for data
WINDOW = 21     # rolling window size to use as predictors
DATE_COL = "Date"

# File path for the equal and value weighted bottom-up portfolio evaluation results
current_directory = os.getcwd()
bu_equal_portfolio_evaluation_path = os.path.join(current_directory, 'Results', f'bu_equal_portfolio_evaluation{WINDOW:.0f}.csv')
bu_value_portfolio_evaluation_path = os.path.join(current_directory, 'Results', f'bu_value_portfolio_evaluation{WINDOW:.0f}.csv')

# File path for the equal and value weighted bottom-up portfolio results
bu_equal_portfolio_results_path = os.path.join(current_directory, 'Results', f'bu_equal_portfolio_results{WINDOW:.0f}.csv')
bu_value_portfolio_results_path = os.path.join(current_directory, 'Results', f'bu_value_portfolio_results{WINDOW:.0f}.csv')

# File path for the equal and value weighted ML-based long-short portfolio results
ml_equal_portfolio_results_path = os.path.join(current_directory, 'Results', f'ml_equal_portfolio_results{WINDOW:.0f}.csv')
ml_value_portfolio_results_path = os.path.join(current_directory, 'Results', f'ml_value_portfolio_results{WINDOW:.0f}.csv')

# File path to save evaluation metrics graphs for equal and value weighted bottom-up portfolio
equal_bu_evaluation_graph_path = os.path.join(current_directory, 'Results', 'Graphs', f'equal_bu_evaluation_graph{WINDOW:.0f}.png')
value_bu_evaluation_graph_path = os.path.join(current_directory, 'Results', 'Graphs', f'value_bu_evaluation_graph{WINDOW:.0f}.png')

# File path to save cumulative growth graph for ML-based equal and value weighted long-short portfolio
equal_cumulative_growth_graph_path = os.path.join(current_directory, 'Results', 'Graphs', f'equal_cumulative_growth_graph{WINDOW:.0f}.png')
value_cumulative_growth_graph_path = os.path.join(current_directory, 'Results', 'Graphs', f'value_cumulative_growth_graph{WINDOW:.0f}.png')

In [ ]:
# Dictionary for model names
models_dict = {"r_bu_portfolio": "Bottom-up",
               "r_ols": "OLS",
               "r_lasso": "Lasso",
               "r_ridge": "Ridge",
               "r_enet": "Elastic Net",
               "r_rf": "RF",
               "r_xgb": "XGB",
               "r_nn1": "NN1",
               "r_nn2": "NN2",
               "r_nn3": "NN3",
               "r_nn4": "NN4",
               "r_nn5": "NN5",
               "r_tfm1": "TimesFM 1.0",
               "r_tfm2": "TimesFM 2.0",
               "r_chr_bolt_tiny": "Chronos-Bolt-Tiny",
               "r_chr_bolt_mini": "Chronos-Bolt-Mini",
               "r_chr_bolt_small": "Chronos-Bolt-Small",
               "r_chr_bolt_base": "Chronos-Bolt-Base",
               "r_chr_t5_tiny": "Chronos-T5-Tiny",
               "r_chr_t5_mini": "Chronos-T5-Mini",
               "r_chr_t5_small": "Chronos-T5-Small",
               "r_moirai_s": "Moirai-Small",
               "r_moirai_moe_s": "Moirai-MoE-Small",
               "r_moirai_moe_b": "Moirai-MoE-Base"
               }

# Dictionary for model type
model_type = {"Linear": ["OLS", "Lasso", "Ridge", "Elastic Net"],
              "Non-Linear": ["RF", "XGB"],
              "Neural Network": ["NN1", "NN2", "NN3", "NN4", "NN5"],
              "TimesFM": ["TimesFM 1.0", "TimesFM 2.0"],
              "Chronos-Bolt": ["Chronos-Bolt-Tiny", "Chronos-Bolt-Mini", "Chronos-Bolt-Small", "Chronos-Bolt-Base"],
              "Chronos-T5": ["Chronos-T5-Tiny", "Chronos-T5-Mini", "Chronos-T5-Small"],
              "Moirai": ["Moirai-Small"],
              "Moirai-MoE": ["Moirai-MoE-Small", "Moirai-MoE-Base"]
              }

In [ ]:
# Colour Map
palette = {"Bottom-up": "black",
           "Linear": "orange",
           "Non-Linear": "red",
           "Neural Network": "green",
           "TimesFM": "brown",
           "Chronos-Bolt": "blue",
           "Chronos-T5": "darkblue",
           "Moirai": "purple",
           "Moirai-MoE": "deeppink"
           }

### Load Data for Visualizations

In [ ]:
# Load the evaluation results for equal and value weighted bottom-up portfolios
bu_equal_portfolio_evaluation_df = pd.read_csv(bu_equal_portfolio_evaluation_path)
bu_value_portfolio_evaluation_df = pd.read_csv(bu_value_portfolio_evaluation_path)

# Load the results for equal and value weighted bottom-up portfolios
bu_equal_portfolio_results_df = pd.read_csv(bu_equal_portfolio_results_path, parse_dates=[DATE_COL], index_col=DATE_COL)
bu_value_portfolio_results_df = pd.read_csv(bu_value_portfolio_results_path, parse_dates=[DATE_COL], index_col=DATE_COL)

# Load the results for equal and value weighted ML-based portfolios
ml_equal_portfolio_results_df = pd.read_csv(ml_equal_portfolio_results_path, parse_dates=[DATE_COL], index_col=DATE_COL)
ml_value_portfolio_results_df = pd.read_csv(ml_value_portfolio_results_path, parse_dates=[DATE_COL], index_col=DATE_COL)

### Plot Economic Evaluation for Bottom-Up Portfolios

In [ ]:
# Add Aodel Type to Bottom-Up Portfolio Evalution Results
def get_model_type(model):
    for k, v in model_type.items():
        if model in v:
            return k

bu_equal_portfolio_evaluation_df["Model Type"] = bu_equal_portfolio_evaluation_df["Model"].apply(get_model_type)
bu_value_portfolio_evaluation_df["Model Type"] = bu_value_portfolio_evaluation_df["Model"].apply(get_model_type)

In [ ]:
# Convert directional accuracy from Percentage to Decimal format and calculate deviation from chance (50%)
bu_equal_portfolio_evaluation_df["DA (Equal)"] = bu_equal_portfolio_evaluation_df["DA (Equal)"] / 100
bu_equal_portfolio_evaluation_df["DA_Centered"] = bu_equal_portfolio_evaluation_df["DA (Equal)"] - 0.5
bu_value_portfolio_evaluation_df["DA (Value)"] = bu_value_portfolio_evaluation_df["DA (Value)"] / 100
bu_value_portfolio_evaluation_df["DA_Centered"] = bu_value_portfolio_evaluation_df["DA (Value)"] - 0.5

In [ ]:
# Plot Evaluation Metrics for Equal-Weighted Bottom-Up Portfolio
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)

sns.barplot(data=bu_equal_portfolio_evaluation_df, x="Model", y="R2 (Equal)", hue="Model Type", palette=palette, dodge=False, ax=ax1)
ax1.set_title("Bottom-up Portfolio: R²")
ax1.set_ylabel("R² (log scale)")
ax1.set_yscale("symlog")
ax1.tick_params(axis='x', rotation=60)
ax1.legend_.remove()

sns.barplot(data=bu_equal_portfolio_evaluation_df, x="Model", y="DA_Centered", hue="Model Type", palette=palette, dodge=False, ax=ax2)
ax2.axhline(0, color="black", linestyle="--", linewidth=1)
ax2.set_title("Bottom-up Portfolio: Directional Accuracy (DA) - Deviation from 50%")
ax2.set_ylabel("Δ DA (%)")
ax2.yaxis.set_major_formatter(lambda y, _: f"{y:.0%}")
ax2.tick_params(axis='x', rotation=60)
ax2.legend_.remove()

plt.tight_layout()
plt.savefig(equal_bu_evaluation_graph_path, dpi=800, bbox_inches="tight")
plt.show()

In [ ]:
# Plot Evaluation Metrics for Value-Weighted Bottom-Up Portfolio
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)

sns.barplot(data=bu_value_portfolio_evaluation_df, x="Model", y="R2 (Value)", hue="Model Type", palette=palette, dodge=False, ax=ax1)
ax1.set_title("Bottom-up Portfolio: R²")
ax1.set_ylabel("R² (log scale)")
ax1.set_yscale("symlog")
ax1.tick_params(axis='x', rotation=60)
ax1.legend_.remove()

sns.barplot(data=bu_value_portfolio_evaluation_df, x="Model", y="DA_Centered", hue="Model Type", palette=palette, dodge=False, ax=ax2)
ax2.axhline(0, color="black", linestyle="--", linewidth=1)
ax2.set_title("Bottom-up Portfolio: Directional Accuracy (DA) - Deviation from 50%")
ax2.set_ylabel("Δ DA (%)")
ax2.yaxis.set_major_formatter(lambda y, _: f"{y:.0%}")
ax2.tick_params(axis='x', rotation=60)
ax2.legend_.remove()

plt.tight_layout()
plt.savefig(value_bu_evaluation_graph_path, dpi=800, bbox_inches="tight")
plt.show()

### Plot Economic Evaluation for Long-Short Portfolios

In [ ]:
# Combine Bottom-Up Portfolio Returns with Long-Short Portfolio Returns to Provide Baseline
ml_ep_results_df = ml_equal_portfolio_results_df.merge(bu_equal_portfolio_results_df["r_bu_portfolio"], how="left", on=DATE_COL).rename(columns=models_dict)
ml_vp_results_df = ml_value_portfolio_results_df.merge(bu_value_portfolio_results_df["r_bu_portfolio"], how="left", on=DATE_COL).rename(columns=models_dict)

In [ ]:
# Set Colour Map for Bottom-Up Portfolio
ml_ep_selected = ["Bottom-up"]
ml_ep_palette = {"Bottom-up": palette["Bottom-up"]}
for type, models in model_type.items():
    cum_rets = (1 + ml_ep_results_df[models]).prod() - 1
    best_model = cum_rets.idxmax()
    ml_ep_selected.append(best_model)
    ml_ep_palette[best_model] = palette[type]

ml_vp_selected = ["Bottom-up"]
ml_vp_palette = {"Bottom-up": palette["Bottom-up"]}
for type, models in model_type.items():
    cum_rets = (1 + ml_vp_results_df[models]).prod() - 1
    best_model = cum_rets.idxmax()
    ml_vp_selected.append(best_model)
    ml_vp_palette[best_model] = palette[type]

In [ ]:
# Calculate Cummulative Growth
ml_equal_portfolio_cum_growth = (1 + ml_ep_results_df[ml_ep_selected]).cumprod() - 1
ml_value_portfolio_cum_growth = (1 + ml_vp_results_df[ml_vp_selected]).cumprod() - 1

In [ ]:
ml_ep_cum_growth_plot_df = ml_equal_portfolio_cum_growth.reset_index().melt(id_vars="Date", var_name="Model", value_name="Growth")
ml_vp_cum_growth_plot_df = ml_value_portfolio_cum_growth.reset_index().melt(id_vars="Date", var_name="Model", value_name="Growth")

In [ ]:
# Plot Compounded Return for Top Model in Each Model Type Based on CAGR for Equal-Weighted Long-Short Portfolios
plt.figure(figsize=(12,8))
ax = sns.lineplot(data=ml_ep_cum_growth_plot_df, x="Date", y="Growth", hue="Model", linewidth=0.9, palette=ml_ep_palette)
plt.axhline(0, color="grey", linestyle="--", linewidth=1.2)
ax.set_title("Compounded Return (Growth)")
ax.set_xlabel("Date")
ax.set_xlim(pd.to_datetime("2015-12-31"), pd.to_datetime("2025-01-01"))
ax.set_ylabel("Growth")
ax.set_ylim(-2, 16)
ax.legend(title="Model", bbox_to_anchor=(0.99, 0.99), loc="upper right", borderaxespad=0)

plt.tight_layout()
plt.savefig(equal_cumulative_growth_graph_path, dpi=800, bbox_inches="tight")
plt.show()

In [ ]:
# Plot Compounded Return for Top Model in Each Model Type Based on CAGR for Value-Weighted Long-Short Portfolios
plt.figure(figsize=(12,8))
ax = sns.lineplot(data=ml_vp_cum_growth_plot_df, x="Date", y="Growth", hue="Model", linewidth=0.9, palette=ml_vp_palette)
plt.axhline(0, color="grey", linestyle="--", linewidth=1.2)
ax.set_title("Compounded Return (Growth)")
ax.set_xlabel("Date")
ax.set_xlim(pd.to_datetime("2015-12-31"), pd.to_datetime("2025-01-01"))
ax.set_ylabel("Growth")
ax.set_ylim(-2, 20)
ax.legend(title="Model", bbox_to_anchor=(0.99, 0.99), loc="upper right", borderaxespad=0)

plt.tight_layout()
plt.savefig(value_cumulative_growth_graph_path, dpi=800, bbox_inches="tight")
plt.show()